# Practice

In [7]:
import numpy as np
import torch


box1 = np.array([1.0, 2.0, 3.0, 4.0])

In [37]:
def calc_iou(box1, box2):
    """
    Param: box1, box2
    Return: Intersection over Union of two boxes
    
    Each boxes should be like [x1, y1, x2, y2],
    and x1 <= x2, y1 <= y2
    """
    
    (ax1, ay1, ax2, ay2) = box1
    (bx1, by1, bx2, by2) = box2
    
    assert (ax1 <= ax2) & (ay1 <= ay2)
    assert (bx1 <= bx2) & (by1 <= by2)
    
    cx1 = max(ax1, bx1)
    cy1 = max(ay1, by1)
    cx2 = min(ax2, bx2)
    cy2 = min(ay2, by2)
    
    assert (cx1 <= cx2) & (cy1 <= cy2)
        
    a_area = (ax2 - ax1) * (ay2 - ay1)
    b_area = (bx2 - bx1) * (by2 - by1)
    c_area = (cx2 - cx1) * (cy2 - cy1)
        
    union_area = a_area + b_area - c_area
    intersecion_area = c_area
    
    smooth = 1e-6
#     print(intersecion_area)
    
    return (intersecion_area + smooth) / (union_area + smooth)

In [99]:
def calc_iou_many_to_one(boxes, ground_truth):
    """
    Param: boxes: shape([N, 4]), ground_truth: shape([4])
    Return: IoU of boxes over on ground truth box
    
    Each boxes should be like [x1, y1, x2, y2],
    and x1 <= x2, y1 <= y2
    """
    
    (gt_x1, gt_y1, gt_x2, gt_y2) = ground_truth
    boxes_x1s = boxes[:, 0]
    boxes_y1s = boxes[:, 1]
    boxes_x2s = boxes[:, 2]
    boxes_y2s = boxes[:, 3]
    
    assert (gt_x1 <= gt_x2) & (gt_y1 <= gt_y2)
    assert (boxes_x1s <= boxes_x2s).all() & (boxes_y1s <= boxes_y2s).all()
    
    inter_x1s = torch.max(boxes_x1s, gt_x1)
    inter_y1s = torch.max(boxes_y1s, gt_y1)
    inter_x2s = torch.min(boxes_x2s, gt_x2)
    inter_y2s = torch.min(boxes_y2s, gt_y2)
    
    assert (inter_x1s <= inter_x2s).all() & (inter_y1s <= inter_y2s).all()
        
    gt_area = (gt_x2 - gt_x1) * (gt_y2 - gt_y1)
    box_areas = (boxes_x2s - boxes_x1s) * (boxes_y2s - boxes_y1s)
    intersect_areas = (inter_x2s - inter_x1s) * (inter_y2s - inter_y1s)
    
    union_area = gt_area + box_areas - intersect_areas
    intersecion_area = intersect_areas

    smooth = 1e-6    
    return (intersecion_area + smooth) / (union_area + smooth)


In [100]:
ground_truth = torch.tensor([2.0, 2.0, 5.0, 5.0])

many_boxes = torch.tensor([
    [1.0, 1.0, 3.0, 3.0],
    [1.0, 4.0, 3.0, 6.0],
    [4.0, 4.0, 6.0, 6.0],
    [2.0, 2.0, 4.0, 4.0],
    [3.0, 3.0, 4.0, 4.0]
])

In [102]:
ret = calc_iou_many_to_one(many_boxes, ground_truth)
ret

tensor([0.0833, 0.0833, 0.0833, 0.4444, 0.1111])

In [103]:
threshold = 0.0001

torch.all(
    torch.lt(
        torch.abs(torch.add(ret,
                            -torch.tensor([1/12, 1/12, 1/12, 4/9, 1/9]))),
    threshold))

tensor(True)

In [104]:
torch.lt(torch.abs(torch.add(ret,-torch.tensor([1/12, 1/12, 1/12, 4/9, 1/9]))),threshold)

tensor([True, True, True, True, True])

In [98]:
(boxes_x1s <= boxes_x2s).all()

tensor(True)

In [91]:
ground_truth.shape

torch.Size([4])

In [92]:
many_boxes.shape

torch.Size([5, 4])

In [63]:
(gt_x1, gt_y1, gt_x2, gt_y2) = ground_truth

In [59]:
many_boxes

tensor([[1., 1., 3., 3.],
        [1., 4., 3., 6.],
        [4., 4., 6., 6.],
        [2., 2., 4., 4.],
        [3., 3., 4., 4.]])

In [64]:
boxes_x1s = many_boxes[:, 0]
boxes_y1s = many_boxes[:, 1]
boxes_x2s = many_boxes[:, 2]
boxes_y2s = many_boxes[:, 3]

In [68]:
gt_x1

tensor(2.)

In [67]:
boxes_x1s

tensor([1., 1., 4., 2., 3.])

In [81]:
inter_x1s = torch.max(boxes_x1s, gt_x1)
inter_y1s = torch.max(boxes_y1s, gt_y1)
inter_x2s = torch.min(boxes_x2s, gt_x2)
inter_y2s = torch.min(boxes_y2s, gt_y2)

In [ ]:
gt_area = (ax2 - ax1) * (ay2 - ay1)

In [74]:
boxes_x2s - boxes_x1s

tensor([2., 2., 2., 2., 1.])

In [82]:
gt_area = (gt_x2 - gt_x1) * (gt_y2 - gt_y1)
box_areas = (boxes_x2s - boxes_x1s) * (boxes_y2s - boxes_y1s)
intersect_areas = (inter_x2s - inter_x1s) * (inter_y2s - inter_y1s)

In [86]:
gt_area

tensor(9.)

In [84]:
gt_area + box_areas

tensor([13., 13., 13., 13., 10.])

In [87]:
union_area = gt_area + box_areas - intersect_areas
intersecion_area = intersect_areas


In [88]:
intersecion_area / union_area

tensor([0.0833, 0.0833, 0.0833, 0.4444, 0.1111])

In [77]:
box_areas

tensor([4., 4., 4., 4., 1.])

In [78]:
gt_area

tensor(9.)

In [65]:
boxes_x1s

tensor([1., 1., 4., 2., 3.])

In [66]:
boxes_y2s

tensor([3., 6., 6., 4., 4.])

In [42]:
box  = torch.tensor([2.0, 2.0, 5.0, 5.0])

In [43]:
box1 = torch.tensor([0.0, 0.0, 2.0, 2.0])

ret = calc_iou(box, box1)

In [47]:
ret.item()

7.692307235629414e-08

In [31]:
ret.shape

torch.Size([])

In [32]:
box.shape

torch.Size([4])

In [33]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)


In [35]:
x_data.shape

torch.Size([2, 2])